In [1]:
# HMDA Lending Bias Analysis
# This script analyzes HMDA data across multiple states to explore lending bias by race.
# It includes data preprocessing, visualizations, regression models, and fairness metrics.import pandas as pd

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import statsmodels.api as sm
from imblearn.over_sampling import SMOTE
# Set up the path to save visualizations
downloads_path = r"C:\Users\chall\OneDrive\Documents"
os.makedirs(downloads_path, exist_ok=True)

In [2]:
# Load and concatenate state-level HMDA data with corrected state assignments
co_data = pd.read_csv(
    r"C:\Users\chall\Downloads\state_CO_actions_taken_1-3-6-7_races_American Indian or Alaska Native-Asian-Black or African America6bc2eb02ff70040dfa7acab9a25eb91a.csv",
    low_memory=False
).assign(state="CO")
mo_data = pd.read_csv(
    r"C:\Users\chall\Downloads\state_MO_actions_taken_1-3-5-6-7_races_American Indian or Alaska Native-Black or African American-Whcf47184be226fb25a036db6bb872aa86.csv",
    low_memory=False
).assign(state="MO")
ca_data = pd.read_csv(
    r"C:\Users\chall\Downloads\state_TX_actions_taken_1-3-5-6-7_races_American Indian or Alaska Native-Black or African American-Wh7ac0aaa84940e60ea778e85a981eaaf4.csv",
    low_memory=False
).assign(state="TX")
tx_data = pd.read_csv(
    r"C:\Users\chall\Downloads\state_California_actions_taken_1-2-3-5-6-7_races_American Indian or Alaska Native-Black or African American-e5491824502a69096e36340f3a48da1a.csv",
    low_memory=False
).assign(state="CA")
df = pd.concat([co_data, mo_data, ca_data, tx_data], ignore_index=True)
print(f"Initial dataset size: {len(df)} rows")
print("Initial columns:", df.columns.tolist())

Initial dataset size: 1133037 rows
Initial columns: ['activity_year', 'lei', 'derived_msa-md', 'state_code', 'county_code', 'census_tract', 'conforming_loan_limit', 'derived_loan_product_type', 'derived_dwelling_category', 'derived_ethnicity', 'derived_race', 'derived_sex', 'action_taken', 'purchaser_type', 'preapproval', 'loan_type', 'loan_purpose', 'lien_status', 'reverse_mortgage', 'open-end_line_of_credit', 'business_or_commercial_purpose', 'loan_amount', 'loan_to_value_ratio', 'interest_rate', 'rate_spread', 'hoepa_status', 'total_loan_costs', 'total_points_and_fees', 'origination_charges', 'discount_points', 'lender_credits', 'loan_term', 'prepayment_penalty_term', 'intro_rate_period', 'negative_amortization', 'interest_only_payment', 'balloon_payment', 'other_nonamortizing_features', 'property_value', 'construction_method', 'occupancy_type', 'manufactured_home_secured_property_type', 'manufactured_home_land_property_interest', 'total_units', 'multifamily_affordable_units', 'inco

In [3]:
# Keep only the columns we need
columns_to_keep = ['income', 'loan_amount', 'action_taken', 'census_tract', 'derived_race', 'state_code', 'state']
df = df[columns_to_keep]

In [4]:
# Drop rows with missing state_code, race, or action_taken
df = df.dropna(subset=['state_code', 'derived_race', 'action_taken'])
df['state_code'] = df['state_code'].astype(str)
df['derived_race'] = df['derived_race'].astype(str)


In [5]:
# Clean up any 'nan' strings
df['state_code'] = df['state_code'].replace('nan', np.nan)
df['derived_race'] = df['derived_race'].replace('nan', np.nan)
df = df.dropna(subset=['state_code', 'derived_race'])

In [6]:
# Map state codes to full names
state_mapping = {"CO": "Colorado", "MO": "Missouri", "CA": "California", "TX": "Texas"}
df['state_code'] = df['state_code'].map(state_mapping)
print("States after mapping:", df['state_code'].unique())

States after mapping: ['Colorado' 'Missouri' 'Texas' 'California']


In [7]:
# Ensure action_taken is numeric
df['action_taken'] = pd.to_numeric(df['action_taken'], errors='coerce')
df = df.dropna(subset=['action_taken'])
df['action_taken'] = df['action_taken'].astype(int)
print("Races:", df['derived_race'].unique())
print("Action taken values:", df['action_taken'].unique())

Races: ['White' 'American Indian or Alaska Native' 'Black or African American'
 'Asian' '2 or more minority races']
Action taken values: [1 3 6 7 5 2]


In [8]:
# Convert income and loan_amount to numeric
df['income'] = pd.to_numeric(df['income'], errors='coerce')
df['loan_amount'] = pd.to_numeric(df['loan_amount'], errors='coerce')
print("Missing income values:", df['income'].isna().sum())
print("Missing loan_amount values:", df['loan_amount'].isna().sum())

Missing income values: 20379
Missing loan_amount values: 0


In [9]:
# Drop states where income or loan_amount are all NaN
state_nan_check = df.groupby('state_code')[['income', 'loan_amount']].apply(lambda x: x.isna().all())
states_to_drop = state_nan_check[(state_nan_check['income'] == True) | (state_nan_check['loan_amount'] == True)].index.tolist()
if states_to_drop:
    print(f"Dropping states with all NaN income/loan_amount: {states_to_drop}")
    df = df[~df['state_code'].isin(states_to_drop)]

In [10]:
# Fill missing income and loan_amount with state-wise medians
df['income'] = df.groupby('state_code')['income'].transform(lambda x: x.fillna(x.median()))
df['loan_amount'] = df.groupby('state_code')['loan_amount'].transform(lambda x: x.fillna(x.median()))
print("Missing income after filling:", df['income'].isna().sum())
print("Missing loan_amount after filling:", df['loan_amount'].isna().sum())

Missing income after filling: 0
Missing loan_amount after filling: 0


In [11]:
# Drop any remaining rows with NaNs
df = df.dropna(subset=['income', 'loan_amount'])
print("Dataset size after cleaning:", len(df))

# Verify income and loan_amount are numeric
if not (df['income'].dtype in [np.float64, np.int64] and df['loan_amount'].dtype in [np.float64, np.int64]):
    raise ValueError("Income or loan_amount is not numeric after cleaning")
print("Income type:", df['income'].dtype)
print("Loan amount type:", df['loan_amount'].dtype)

Dataset size after cleaning: 1133037
Income type: float64
Loan amount type: float64


In [12]:
# Convert action_taken to binary: 1 = approved, 0 = denied
df['action_taken'] = df['action_taken'].apply(lambda x: 1 if x == 1 else 0)
print("Action taken after mapping:", df['action_taken'].unique())


Action taken after mapping: [1 0]


In [13]:
# Simplify race categories
df = df[~df['derived_race'].isin(['Asian', 'Joint', 'Free Form Text Only', 'Race Not Available'])]
df.loc[df['derived_race'].isin(['American Indian or Alaska Native', 'Native Hawaiian or Other Pacific Islander']), 'derived_race'] = 'Other'
print(f"Dataset size after race simplification: {len(df)} rows")
print("Races after simplification:", df['derived_race'].unique())

Dataset size after race simplification: 1126535 rows
Races after simplification: ['White' 'Other' 'Black or African American' '2 or more minority races']


In [14]:
# Flag high-minority census tracts
df['census_tract'] = df['census_tract'].astype(str)
df['high_minority_tract'] = (df['census_tract'].str[-4:].astype(float) > 5000).astype(int)
print(f"High-minority tracts: {df['high_minority_tract'].sum()}")


High-minority tracts: 0


In [15]:
# Approval Rates by Race and State
df['approved'] = df['action_taken']
approval_rates = df.groupby(['state_code', 'derived_race'])['approved'].mean().reset_index()

plt.figure(figsize=(12, 6))
sns.barplot(x='state_code', y='approved', hue='derived_race', data=approval_rates, palette=['tab:blue', 'tab:orange', 'lightgray', 'darkgray'])
plt.title('Loan Approvals by Race and State', fontsize=20)
plt.xlabel('State', fontsize=18)
plt.ylabel('Approval Rate', fontsize=20)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
# Place legend on the left side
plt.legend(title='Race', fontsize=14, title_fontsize=14, loc='center left', bbox_to_anchor=(-0.3, 0.5))
plt.subplots_adjust(left=0.25, right=0.95, top=0.9, bottom=0.1)
plt.savefig(os.path.join(downloads_path, 'loan_approvals_by_race_state.png'))
plt.close()

In [16]:
# Income and Loan Amount by Approval Status
combined_means = df.groupby(['state_code', 'approved'])[['income', 'loan_amount']].mean().reset_index()

fig, ax1 = plt.subplots(figsize=(14, 6))
# Income on left y-axis
sns.barplot(x='state_code', y='income', hue='approved', data=combined_means, palette=['tab:blue', 'tab:orange'], ax=ax1, alpha=0.8, width=0.4)
ax1.set_xlabel('State', fontsize=18)
ax1.set_ylabel('Average Income', fontsize=18, color='tab:blue', labelpad=20)
ax1.tick_params(axis='y', labelcolor='tab:blue', labelsize=14)
ax1.set_title('Average Income and Loan Amount by Approval Status Across States', fontsize=20, pad=25)

# Loan amount on right y-axis
ax2 = ax1.twinx()
sns.barplot(x='state_code', y='loan_amount', hue='approved', data=combined_means, palette=['tab:blue', 'tab:orange'], ax=ax2, alpha=0.4, width=0.4)
ax2.set_ylabel('Average Loan Amount', fontsize=18, color='tab:orange', labelpad=20)
ax2.tick_params(axis='y', labelcolor='tab:orange', labelsize=14)

# Manage legends to avoid duplicates and place on the left
ax1.get_legend().remove()
handles, labels = ax2.get_legend_handles_labels()
fig.legend(handles[:2], ['Approved (Yes)', 'Denied (No)'], title='Approval Status', fontsize=14, title_fontsize=14, 
           loc='center left', bbox_to_anchor=(-0.3, 0.5))
ax2.get_legend().remove()

plt.xticks(fontsize=14)
plt.subplots_adjust(left=0.25, right=0.85, top=0.9, bottom=0.1)
plt.savefig(os.path.join(downloads_path, 'income_and_loan_amount_by_approval.png'))
plt.close()

In [17]:
# OLS Regression for Approval Analysis
df_ols = df.copy()
df_ols['income'] = pd.to_numeric(df_ols['income'], errors='coerce')
df_ols['loan_amount'] = pd.to_numeric(df_ols['loan_amount'], errors='coerce')
df_ols['action_taken'] = df_ols['action_taken'].astype(int)
df_ols['high_minority_tract'] = df_ols['high_minority_tract'].astype(int)

df_ols = df_ols.dropna(subset=['income', 'loan_amount'])
print("Dataset size for OLS:", len(df_ols))


Dataset size for OLS: 1126535


In [18]:
# Create dummy variables for race and state
df_ols = pd.get_dummies(df_ols, columns=['derived_race', 'state_code'], drop_first=True, dtype=np.uint8)
dummy_cols = [col for col in df_ols.columns if col.startswith('derived_race_') or col.startswith('state_code_')]
for col in dummy_cols:
    df_ols[col] = pd.to_numeric(df_ols[col], errors='coerce').astype(np.uint8)


In [19]:
# Prepare features for OLS
features = ['income', 'loan_amount', 'high_minority_tract'] + dummy_cols
X_ols = df_ols[features]
X_ols = sm.add_constant(X_ols)
y_ols = df_ols['action_taken']


In [20]:
# Run OLS regression
ols_model = sm.OLS(y_ols, X_ols).fit()
print("OLS Results:")
print(ols_model.summary())

OLS Results:
                            OLS Regression Results                            
Dep. Variable:           action_taken   R-squared:                       0.025
Model:                            OLS   Adj. R-squared:                  0.025
Method:                 Least Squares   F-statistic:                     3591.
Date:                Sun, 04 May 2025   Prob (F-statistic):               0.00
Time:                        16:50:48   Log-Likelihood:            -7.3233e+05
No. Observations:             1126535   AIC:                         1.465e+06
Df Residuals:                 1126526   BIC:                         1.465e+06
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------

In [21]:
# Logistic Regression and Bias Analysis
df_lr = df.copy()
df_lr['income'] = pd.to_numeric(df_lr['income'], errors='coerce')
df_lr['loan_amount'] = pd.to_numeric(df_lr['loan_amount'], errors='coerce')
df_lr['action_taken'] = df_lr['action_taken'].astype(int)
df_lr['high_minority_tract'] = df_lr['high_minority_tract'].astype(int)

df_lr = df_lr.dropna(subset=['income', 'loan_amount'])
print("Dataset size for logistic regression:", len(df_lr))

Dataset size for logistic regression: 1126535


In [24]:
# Create dummy variables
df_lr = pd.get_dummies(df_lr, columns=['derived_race', 'state_code'], drop_first=True, dtype=np.uint8)
dummy_cols_lr = [col for col in df_lr.columns if col.startswith('derived_race_') or col.startswith('state_code_')]
for col in dummy_cols_lr:
    df_lr[col] = pd.to_numeric(df_lr[col], errors='coerce').astype(np.uint8)


In [ ]:
# Prepare features and target
features_lr = ['income', 'loan_amount', 'high_minority_tract'] + dummy_cols_lr
X = df_lr[features_lr]
y = df_lr['action_taken']


In [ ]:
# Split data, ensuring race stratification
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=df.loc[X.index, 'derived_race'])
print("Race distribution in test set:", df.loc[X_test.index, 'derived_race'].value_counts())


In [ ]:
# Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)


In [ ]:
# Train logistic regression model
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train_smote, y_train_smote)
y_pred = model.predict(X_test)

print("Classification Report:")
print(classification_report(y_test, y_pred))

In [ ]:
# Confusion matrix visualization
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap=sns.color_palette(['tab:blue', 'tab:red'], as_cmap=True),
            xticklabels=["Denied", "Approved"], yticklabels=["Denied", "Approved"])
plt.title('Model Prediction Accuracy (Confusion Matrix)', fontsize=16)
plt.xlabel('Predicted', fontsize=14)
plt.ylabel('Actual', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()
plt.savefig(os.path.join(downloads_path, "model_prediction_accuracy.png"))
plt.close()

In [ ]:
# Compare actual vs predicted approval rates by race
X_test_reset = X_test.reset_index(drop=True)
y_test_reset = pd.Series(y_test, name='action_taken').reset_index(drop=True)
y_pred_reset = pd.Series(y_pred, name='predicted').reset_index(drop=True)
test_indices = X_test.index
race_data = df.loc[test_indices, 'derived_race'].reset_index(drop=True)

results_df = pd.DataFrame({
    'derived_race': race_data,
    'actual': y_test_reset,
    'predicted': y_pred_reset
})

actual_rates = results_df.groupby('derived_race')['actual'].mean().reset_index()
actual_rates.columns = ['derived_race', 'actual_approval_rate']
predicted_rates = results_df.groupby('derived_race')['predicted'].mean().reset_index()
predicted_rates.columns = ['derived_race', 'predicted_approval_rate']
approval_rates_df = pd.merge(actual_rates, predicted_rates, on='derived_race')

approval_rates_melted = approval_rates_df.melt(id_vars='derived_race',
                                               value_vars=['actual_approval_rate', 'predicted_approval_rate'],
                                               var_name='Rate Type', value_name='Approval Rate')
plt.figure(figsize=(10, 6))
sns.barplot(x='derived_race', y='Approval Rate', hue='Rate Type', data=approval_rates_melted, palette=['tab:blue', 'tab:orange'])
plt.title('Actual vs Predicted Loan Approvals by Race', fontsize=16)
plt.xlabel('Race', fontsize=14)
plt.ylabel('Approval Rate', fontsize=14)
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)
# Place legend on the left side
plt.legend(title='Rate Type (Actual vs Predicted)', fontsize=14, title_fontsize=14, loc='center left', bbox_to_anchor=(-0.3, 0.5))
plt.subplots_adjust(left=0.25, right=0.95, top=0.9, bottom=0.2)
plt.savefig(os.path.join(downloads_path, 'actual_vs_predicted_approvals_by_race.png'))
plt.close()


In [ ]:
# Fairness Metrics
def calculate_eod(df, group_col, actual_col, pred_col, groups):
    tpr_diff = 0
    fpr_diff = 0
    for group in groups:
        group_data = df[df[group_col] == group]
        tp = len(group_data[(group_data[actual_col] == 1) & (group_data[pred_col] == 1)])
        fn = len(group_data[(group_data[actual_col] == 1) & (group_data[pred_col] == 0)])
        tpr = tp / (tp + fn) if (tp + fn) > 0 else 0
        fp = len(group_data[(group_data[actual_col] == 0) & (group_data[pred_col] == 1)])
        tn = len(group_data[(group_data[actual_col] == 0) & (group_data[pred_col] == 0)])
        fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
        df.loc[df[group_col] == group, 'TPR'] = tpr
        df.loc[df[group_col] == group, 'FPR'] = fpr
    tpr_diff = df['TPR'].max() - df['TPR'].min()
    fpr_diff = df['FPR'].max() - df['FPR'].min()
    return max(tpr_diff, fpr_diff)

racial_groups = ['White', 'Black or African American', '2 or more minority races', 'Other']
eod = calculate_eod(results_df, 'derived_race', 'actual', 'predicted', racial_groups)
print(f"Equalized Odds Difference (EOD): {eod:.4f}")

In [ ]:
# Calculate White-Black approval gap
if ('White' in approval_rates_df['derived_race'].values) and ('Black or African American' in approval_rates_df['derived_race'].values):
    white_pred_rate = approval_rates_df[approval_rates_df['derived_race'] == 'White']['predicted_approval_rate'].iloc[0]
    black_pred_rate = approval_rates_df[approval_rates_df['derived_race'] == 'Black or African American']['predicted_approval_rate'].iloc[0]
    white_black_gap = white_pred_rate - black_pred_rate
    print(f"White-Black Predicted Approval Rate Gap: {white_black_gap:.4f}")
else:
    print("Cannot calculate White-Black gap: Missing racial groups.")
    print("Races in results:", approval_rates_df['derived_race'].unique())